# University Course Brochure Generator

**Business Case**

Build an AI chatbot that builds a Brochure for a university and the programs it offer to prospective students, just by providing the name of the university and its primary website.

This is based on the materials in LLM Engineering course (https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/)

**Technical Challenges**

- To use multi-shot prompting in making links to provide examples of what a program should look like (e.g. undergrad, postgrad)
- Add more instructions to provide the brochure in a particular format (for example, targeted to undergraduates versus experienced professionals looking for a career change versus academics who want to get PHDs)
- Add additional language capabilities (e.g. translate the entire brochure in different language than English)

This will be generated using OpenAI GPT 4-o mini model.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


Below is a class to represent a website that is to be scraped. In addition to scraping the website, the class will also get the links and store the links inside links object.

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
# example using my own website
zkh = Website("https://zinkohlaing.com")
zkh.links

['https://www.zinkohlaing.com',
 'https://www.zinkohlaing.com/',
 'https://www.zinkohlaing.com/about/',
 'https://www.zinkohlaing.com/tag/data-science/',
 '#/portal/signin',
 '#/portal/signup',
 '/comparing-local-and-openai-models-for-website-summarization/',
 '/hello-from-my-new-blog/',
 '/my-experience-with-georgia-tech-omsa-program-part-4/',
 '/my-experience-with-georgia-tech-omsa-program-part-3/',
 '/my-experience-with-georgia-tech-omsa-program-part-2/',
 '/my-experience-with-georgia-tech-omsa-program-part-1/',
 'https://ghost.org/']

In [5]:
# example of how the object contents look like
print(zkh.get_contents())

Webpage Title:
A blog by ZinKo
Webpage Contents:
A blog by ZinKo
Home
About
Data Science Portfolio
Sign in
Subscribe
Thoughts, stories and ideas.
Subscribe below to receive my latest posts directly in your inbox. Promise no spams!
jamie@example.com
Subscribe
Comparing Local and OpenAI Models for Website Summarization
Feb 27, 2025
4 min read
Hello from my new blog!
Nov 8, 2024
1 min read
My Experience with Georgia Tech OMSA Program Part-4
May 1, 2024
6 min read
My Experience with Georgia Tech OMSA Program Part-3
Apr 30, 2024
3 min read
My Experience with Georgia Tech OMSA Program Part-2
Apr 29, 2024
4 min read
My Experience with Georgia Tech OMSA Program Part-1
Mar 25, 2024
3 min read
A blog by ZinKo © 2025. Powered by
Ghost




**First step: Have GPT-4o-mini figure out which links are relevant**

It's good to scrape the website and get the links, but not all links are relevant. 
For this use case, the GPT-4o-mini should be able to decide which links are relevant for a university, and replace relative links such as "/programmes" with "https://exampleuni.com/programmes".

I will use "multi-shot prompting" in which I will provide an example of how it should responsd in the prompt. 

In [14]:
link_system_prompt = "You are an AI assistant specialized in analyzing webpage links for university brochures. \
Your task is to identify and categorize the most relevant links that should be included in a university brochure. \
Consider the following link types as highly relevant: \
1. Program pages (e.g., undergraduate, graduate, post-graduate, diplomas, specific majors) \
2. Admissions pages (e.g., application process, requirements, deadlines) \
3. Campus life pages (e.g., housing, student organizations, events) \
4. Financial aid and scholarship information \
5. Research opportunities and facilities \
6. Career services and alumni success stories \
7. International student resources \
8. Virtual tours or campus maps \
\
Exclude links that are not directly relevant to prospective students, such as: \
- Staff-only pages \
- Technical support or IT services \
- Alumni pages \
- Detailed policy pages, Terms of Services or Privacy Policy pages \
- News articles \
- Library links \
- Career pages (for staff hires) \
n"
link_system_prompt += "Analyze the provided links and you should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "program page", "url": "https://university.url/programs/computer-science"},
        {"type": "admissions page", "url": "https://another.university.url/admissions/apply"},
        {"type": "campus life", "url": "https://another.university.url/campus-life/housing"},
        {"type": "program page", "url": "https://another.university.url/programs/computer-science"}
    ]
}
"""

In [15]:
print(link_system_prompt)

You are an AI assistant specialized in analyzing webpage links for university brochures. Your task is to identify and categorize the most relevant links that should be included in a university brochure. Consider the following link types as highly relevant: 1. Program pages (e.g., undergraduate, graduate, post-graduate, diplomas, specific majors) 2. Admissions pages (e.g., application process, requirements, deadlines) 3. Campus life pages (e.g., housing, student organizations, events) 4. Financial aid and scholarship information 5. Research opportunities and facilities 6. Career services and alumni success stories 7. International student resources 8. Virtual tours or campus maps Exclude links that are not directly relevant to prospective students, such as: - Staff-only pages - Technical support or IT services - Alumni pages - Detailed policy pages, Terms of Services or Privacy Policy pages - News articles - Library links - Career pages (for staff hires) nAnalyze the provided links and 

In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the university, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
# let's try with my school Gtech site
gtech = Website("https://www.gatech.edu/")

In [18]:
print(get_links_user_prompt(gtech))

Here is the list of links on the website of https://www.gatech.edu/ - please decide which of these are relevant web links for a brochure about the university, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-navigation
#main-content
/about
/about/history-traditions
/about/campus-location
/about/rankings
https://www.gatech.edu/about/vision
https://www.gatech.edu/about/values
https://generalcounsel.gatech.edu/ethics-and-compliance
/about/military-resources
/about/visit
/about/leadership
https://directory.gatech.edu/
https://hr.gatech.edu/employment/careers
/offices-and-departments
/academics-education
/academics/all-degree-programs
/academics/colleges-and-schools
https://pe.gatech.edu/
https://pe.gatech.edu/georgia-tech-online
https://registrar.gatech.edu/calendar
/prospective-students
/admission
https://admission.gatech.edu/visit/
https://finaid.gatech.edu/current-cost-overview
https://fina

In [19]:
# calling openAI model
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} #this will only work if you mention in the prompt explicitly that you need the model to return JSON object
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [20]:
# scrape Gtech site
get_links("https://www.gatech.edu/")

{'links': [{'type': 'about page',
   'url': 'https://www.gatech.edu/about/vision'},
  {'type': 'about page', 'url': 'https://www.gatech.edu/about/values'},
  {'type': 'campus life', 'url': 'https://www.gatech.edu/life/campus-life'},
  {'type': 'student activities',
   'url': 'https://www.gatech.edu/life/student-activities'},
  {'type': 'program page',
   'url': 'https://www.gatech.edu/academics/all-degree-programs'},
  {'type': 'admissions page', 'url': 'https://www.gatech.edu/admissions'},
  {'type': 'admissions page', 'url': 'https://www.gatech.edu/admission'},
  {'type': 'financial aid', 'url': 'https://finaid.gatech.edu/'},
  {'type': 'financial aid',
   'url': 'https://finaid.gatech.edu/current-cost-overview'},
  {'type': 'research opportunities', 'url': 'https://research.gatech.edu/'},
  {'type': 'research initiatives',
   'url': 'https://research.gatech.edu/strategic-initiatives'},
  {'type': 'career services', 'url': 'https://career.gatech.edu/'},
  {'type': 'virtual tours', 'u

In [22]:
#try another one 
get_links("https://www.smu.edu/")

{'links': [{'type': 'program page',
   'url': 'https://www.smu.edu/cox/business-degrees/undergraduate/bachelor-business-administration/curriculum/accounting'},
  {'type': 'program page', 'url': 'https://www.smu.edu/meadows/areasofstudy'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/dedman/academics/undergraduate-minors-and-programs/interdisciplinary-studies/biochemistry'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/dedman/academics/departments/chemistry/'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/meadows/areasofstudy/dance/undergraduatestudies/danceperformancebfa'},
  {'type': 'program page', 'url': 'https://www.smu.edu/lyle/undergraduate'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/dedman/academics/departments/history'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/dedman/academics/undergraduate-minors-and-programs/interdisciplinary-studies/ehrp/'},
  {'type': 'program page',
   'url': 'https://www.smu.edu/ded

In [23]:
get_links("https://www.ntu.edu.sg/")

{'links': [{'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/undergraduate'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/undergraduate/admission-guide'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/undergraduate/scholarships'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/undergraduate/financial-matters'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/graduate'},
  {'type': 'admissions page',
   'url': 'https://www.ntu.edu.sg/admissions/graduate/financialmatters'},
  {'type': 'program page',
   'url': 'https://www.ntu.edu.sg/education/colleges-schools'},
  {'type': 'research opportunities', 'url': 'https://www.ntu.edu.sg/research'},
  {'type': 'campus life', 'url': 'https://www.ntu.edu.sg/life-at-ntu'},
  {'type': 'campus life',
   'url': 'https://www.ntu.edu.sg/life

**Observations:**

The agent can't seem to scrape some of the universities websites (such as https://www.ntu.edu.sg/ or https://www.smu.edu.sg/). I suspect this is due to how the site is structured.



# Make the university brochure!

Here's what is happening now.

First I will use the function built above to generate the website and relevant links. 
And then I will iterate each link inside the links to get the content of the link and append them into an object named result.
Finally, the result will be returned.

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [30]:
#print(get_all_details("https://www.gatech.edu/"))

That returned a rather large object which I will comment out later to save you from having to read through the entire thing!

In [31]:
# writing the system prompt to generate the university brochure
# here, I will write the prompt in a way that it will generate the brochure in a style that I wanted.

system_prompt = "You are an AI assistant that analyzes the contents of several relevant pages from a university website. \
and creates a short brochure about the university for prospective students. Respond in markdown.\
Write the brochure by starting with the introduction to the university, Campus Life. \
And then follow up with information for prospective students for undergraduate programs, admission requirements, financial aids. \
Then follow up with postgrad and other programs for experienced professionals. \
Conclude with student resources such as virtual tours and highlight why this is the best choice students can make."

In [32]:
# writing user prompt
def get_brochure_user_prompt(uni_name, url):
    user_prompt = f"You are looking at a university called: {uni_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the university in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [33]:
get_brochure_user_prompt("Harvard", "https://www.harvard.edu/")

Found links: {'links': [{'type': 'program page', 'url': 'https://www.harvard.edu//programs/?degree_levels=undergraduate'}, {'type': 'program page', 'url': 'https://www.harvard.edu//programs/?degree_levels=graduate'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/campus/athletics/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/campus/museums/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/visit/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/visit/tours/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/visit/maps-directions/'}, {'type': 'admissions page', 'url': 'https://college.harvard.edu/'}, {'type': 'financial aid', 'url': 'https://www.harvard.edu/in-focus/'}]}


"You are looking at a university called: Harvard\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the university in markdown.\nLanding page:\nWebpage Title:\nHarvard University\nWebpage Contents:\nSkip to main content\nHarvard University\nSearch\nSearch\nQuick Links\nA to Z index\nMenu\nAcademics\nAcademics\nAcademics\nLearning at Harvard can happen for every type of learner, at any phase of life.\nDegree programs\nAcademics\nDegree programs\nBrowse all of our undergraduate concentrations and  graduate degrees.\nUndergraduate Degrees\nGraduate Degrees\nOther\nProfessional and Lifelong Learning\nHarvard Online\nHarvard Schools\nAcademics\nVisit each School for information on admissions and financial aid.\nExplore more\nHarvard College\nHarvard Business School\nHarvard Division of Continuing Education\nHarvard Divinity School\nHarvard Faculty of Arts and Sciences\nHarvard Kenneth C. Griffin Graduate School of Arts and 

In [34]:
# THE function to create the brochure
def create_brochure(uni_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(uni_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [36]:
create_brochure("Georgia Institute of Technology", "https://www.gatech.edu/")

Found links: {'links': [{'type': 'about', 'url': 'https://www.gatech.edu/about/vision'}, {'type': 'about', 'url': 'https://www.gatech.edu/about/values'}, {'type': 'campus life', 'url': 'https://www.gatech.edu/life/campus-life'}, {'type': 'student activities', 'url': 'https://www.gatech.edu/life/student-activities'}, {'type': 'program page', 'url': 'https://www.gatech.edu/academics/all-degree-programs'}, {'type': 'admissions page', 'url': 'https://www.gatech.edu/admission'}, {'type': 'admissions page', 'url': 'https://www.gatech.edu/admissions'}, {'type': 'visit page', 'url': 'https://admission.gatech.edu/visit/'}, {'type': 'financial aid', 'url': 'https://finaid.gatech.edu/'}, {'type': 'financial aid', 'url': 'https://finaid.gatech.edu/current-cost-overview'}, {'type': 'research opportunities', 'url': 'https://research.gatech.edu/'}, {'type': 'career services', 'url': 'https://career.gatech.edu/'}, {'type': 'international student resources', 'url': 'https://ea.oie.gatech.edu/'}, {'type

# Georgia Institute of Technology Brochure

Welcome to the Georgia Institute of Technology, commonly known as Georgia Tech! Located in the vibrant heart of Atlanta, Georgia, our university is renowned for its commitment to developing exceptional leaders who advance technology and improve the human condition. With a strong emphasis on innovation and research, Georgia Tech strives to transform tomorrow while giving students the tools they need to succeed today.

## Campus Life

At Georgia Tech, campus life is dynamic and engaging. Students enjoy a rich array of activities and experiences, from diverse student organizations to thrilling athletics. Explore the main campus, indulge in the lively traditions, and participate in events throughout the year. Whether you’re a part of the marching band, a club sports team, or one of our 400+ student organizations, there’s something for everyone. Join the cheering crowds at Bobby Dodd Stadium for football games or participate in intramural sports that promote fun and fitness. Georgia Tech fosters a supportive community that encourages personal growth and lifelong friendships.

## Information for Prospective Students

### Undergraduate Programs

Georgia Tech offers a variety of undergraduate programs across several disciplines, including engineering, sciences, business, and design. With a commitment to academic rigor and hands-on experience, students are well-prepared for their careers. 

### Admission Requirements

To apply to Georgia Tech, prospective students must submit an application, including standardized test scores, transcripts, and letters of recommendation. The admission process is competitive, focusing on academic performance, extracurricular involvement, and personal essays. Detailed requirements can be found on our admissions website.

### Financial Aid & Scholarships

Georgia Tech provides numerous financial aid opportunities to make education accessible to all. Scholarships, grants, and loans are available based on need and merit. Students are encouraged to apply for federal and state financial aid and explore scholarship options specific to their intended program.

## Postgraduate and Professional Programs

Georgia Tech also offers a wide range of postgraduate programs, including master’s degrees and certificates aimed at experienced professionals looking to advance their careers. These programs are designed for flexibility, accommodating full-time professionals who wish to continue their education, either on campus or through online learning options.

## Student Resources

We understand that a supportive environment enhances educational success. Georgia Tech offers various resources, including virtual tours for prospective students, academic advising, counseling services, and career support. Our online platforms provide a glimpse into campus life, academic programs, and the vibrant community awaiting you.

### Why Choose Georgia Tech?

At Georgia Tech, students are not just part of a university; they are part of a community that values innovation, collaboration, and leadership. Ranked as the No. 9 public university, Georgia Tech is committed to producing graduates who are equipped to tackle the challenges of tomorrow. Join us in shaping the future—choose Georgia Tech!

--- 

For more information or to schedule a visit, please visit our website or contact the admissions office. We look forward to welcoming you to the Georgia Tech family!

In [37]:
create_brochure("Harvard University", "https://www.harvard.edu/")

Found links: {'links': [{'type': 'program page', 'url': 'https://www.harvard.edu//programs/?degree_levels=undergraduate'}, {'type': 'program page', 'url': 'https://www.harvard.edu//programs/?degree_levels=graduate'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/campus/athletics/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/campus/'}, {'type': 'campus life', 'url': 'https://www.harvard.edu/campus/museums/'}, {'type': 'admissions page', 'url': 'https://college.harvard.edu/'}, {'type': ' admissions page', 'url': 'https://www.harvard.edu/visit/'}, {'type': 'virtual tour', 'url': 'https://www.harvard.edu/visit/tours/'}, {'type': 'virtual tour', 'url': 'https://www.harvard.edu/visit/maps-directions/'}, {'type': 'financial aid', 'url': 'https://www.harvard.edu/about/diversity-and-inclusion/'}, {'type': 'program page', 'url': 'https://pll.harvard.edu/course/cs50-introduction-computer-science'}, {'type': 'research opportunities', 'url': 'https://www.harvard.edu/in-focus

# Harvard University Brochure

## Welcome to Harvard University

Harvard University, located in Cambridge, Massachusetts, is a prestigious institution known for its commitment to academic excellence and research innovation. Established in 1636, Harvard stands as one of the oldest and most respected universities in the world, offering a diverse array of degree programs for learners at every stage of life.

---

## Campus Life

At Harvard, campus life is vibrant and engaging, providing a dynamic atmosphere for personal and academic growth. The university boasts a stunning campus complete with extensive libraries, museums, and a diverse set of extracurricular activities. From athletics to arts, students have a plethora of opportunities to explore their interests and engage with the community.

**Key Features:**
- **Libraries:** Over 70 libraries, including the renowned Widener Library and Houghton Library.
- **Museums:** Access to world-class museums such as the Harvard Art Museums and the Peabody Museum of Archaeology and Ethnology.
- **Athletics:** Join one of Harvard’s many sports teams or enjoy recreational activities within the community.

---

## Undergraduate Programs

Harvard offers a myriad of undergraduate concentrations which cater to a wide range of interests. From the humanities to sciences, students are encouraged to explore their passions deeply and broadly.

### Admission Requirements
Prospective undergraduate students should prepare for a rigorous admission process, which includes:
- Completing the Common Application
- Submitting standardized test scores (SAT or ACT) (Please check university policy for test-optional updates)
- Providing high school transcripts and letters of recommendation

---

## Financial Aid

Harvard is committed to making education accessible. The university offers need-blind admissions for domestic students, ensuring that financial circumstances do not hinder applicants. Financial aid packages are tailored to meet every student's need, with over 70% of undergraduates receiving financial assistance.

---

## Graduate and Professional Programs

Harvard provides a multitude of graduate degrees and professional programs, including:
- **Harvard Business School** 
- **Harvard Law School** 
- **Harvard Medical School**
- **Harvard Kennedy School**

These programs are designed for experienced professionals seeking to advance their careers or specialize further in their fields.

---

## Student Resources

Harvard offers an array of resources to assist both prospective and current students:
- **Virtual Tours:** Explore the beautiful campus from the comfort of your home.
- **Career Services:** Attend workshops and network with industry leaders.
- **Student Organizations:** Engage in over 400 student-run organizations, enhancing both your academic and social experience.

---

## Why Choose Harvard?

Choosing Harvard University is about embracing a legacy of knowledge, innovation, and excellence. With a diverse international community, unmatched resources, and a commitment to developing leaders for the global stage, Harvard is not just a place to learn; it's a place to thrive.

Join us at Harvard and unlock your potential today!

# Generate the brochure in entirely new language!

Now, I will modify the system prompt a bit to generate the brochure in Korean!

In [38]:
kr_system_prompt = "You are an AI assistant that analyzes the contents of several relevant pages from a university website. \
and creates a short brochure about the university for prospective students. Respond in markdown.\
Generate the entire brochure in Korean language. \
Write the brochure by starting with the introduction to the university, Campus Life. \
And then follow up with information for prospective students for undergraduate programs, admission requirements, financial aids. \
Then follow up with postgrad and other programs for experienced professionals. \
Conclude with student resources such as virtual tours and highlight why this is the best choice students can make."

In [39]:
# update the function to create the brochure
def create_brochure_in_kr(uni_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": kr_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(uni_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [40]:
create_brochure_in_kr("Georgia Institute of Technology", "https://www.gatech.edu/")

Found links: {'links': [{'type': 'program page', 'url': 'https://www.gatech.edu/academics/all-degree-programs'}, {'type': 'admissions page', 'url': 'https://www.gatech.edu/admission'}, {'type': 'admissions page', 'url': 'https://admission.gatech.edu/visit/'}, {'type': 'financial aid', 'url': 'https://finaid.gatech.edu/current-cost-overview'}, {'type': 'financial aid', 'url': 'https://finaid.gatech.edu/'}, {'type': 'campus life', 'url': 'https://www.gatech.edu/life/campus-life'}, {'type': 'campus life', 'url': 'https://www.gatech.edu/life/student-activities'}, {'type': 'research opportunities', 'url': 'https://research.gatech.edu/'}, {'type': 'career services', 'url': 'https://career.gatech.edu/'}, {'type': 'international student resources', 'url': 'https://ea.oie.gatech.edu/'}, {'type': 'virtual tour', 'url': 'https://map.gatech.edu/'}]}


# 조지아 공과대학교 브로셔

## 대학 소개
조지아 공과대학교(Georgia Institute of Technology, Georgia Tech)는 세계적인 공학 및 기술 교육 기관으로, 뛰어난 연구 환경과 혁신적인 학문을 통해 인재를 양성하고 있습니다. 조지아주 애틀랜타에 위치한 이 대학은 다채로운 전공과 프로그램을 통해 학생들에게 글로벌 리더로 성장할 기회를 제공합니다.

## 캠퍼스 생활
조지아 공과대학교의 캠퍼스 생활은 활기차고 다양합니다. 학생들은 다양한 학생 활동에 참여하고, 다채로운 스포츠 이벤트에 열기를 더하며, 학술 연구와 사회 활동에 적극적으로 임하고 있습니다. 학생들은 새로운 친구를 만나고, 특별한 경험을 통해 개인적 성장을 이루어나갈 수 있습니다.

## 예비 학생을 위한 정보

### 학부 프로그램
조지아 공과대학교는 공학, 디자인, 컴퓨터 과학, 경영학 등 다양한 전공을 제공하여 각 학생의 흥미와 목표에 맞는 교육을 실시합니다.

### 입학 요건
입학 요건은 전공에 따라 다를 수 있으며, 전반적인 학업 성적, 표준화 시험 점수, 추천서 및 자기소개서를 포함합니다. 자세한 사항은 학교의 공식 웹사이트를 참조하시기 바랍니다.

### 재정 지원
재정 지원 및 장학금 기회도 풍부하여, 수업료와 관련 비용을 보다 쉽게 해결할 수 있도록 돕고 있습니다. 학생들은 또한 다양한 장학금 프로그램에 신청하여 재정적 부담을 줄일 수 있습니다.

## 대학원의 전문 프로그램
조지아 공과대학교는 경력 있는 전문가들을 위한 다양한 석사 및 박사 과정도 운영하고 있습니다. 이러한 프로그램은 고급 기술 및 전문성을 지속적으로 개발하고자 하는 임직원에게 적합합니다.

## 학생 자원
조지아 공과대학교에서는 가상 캠퍼스 투어를 제공하여, 학생들이 대학의 시설과 환경을 사전에 경험할 수 있도록 하고 있습니다. 더불어, 각종 자료와 지원 서비스를 통해 학생들의 성공적인 학업 생활을 지원합니다.

## 최고의 선택
조지아 공과대학교는 세계 수준의 교육, 혁신적인 캠퍼스 생활, 강력한 네트워크와 커뮤니티를 갖춘 곳입니다. 여러분의 미래를 위해 최고의 선택이 될 것입니다. 오늘 조지아 공과대학교에 지원하여 새로운 시작을 만들어보세요!

**Final remarks:**

There you have it! This is how you can use OpenAI or AI agents to easily produce a marketing brochure for a university in a relatively short amount of time!!